In [ ]:
# Imports
from IPython.display import display, HTML

def setup_display(width=95, fontsize=16):
    """
    Sets window width and markdown fontsize for Jupyter notebook. Width is % of window.
    """
    display(HTML("<style>.container { width:"+str(width)+"% !important; }</style>"))
    display(HTML("<style>.rendered_html { font-size: "+str(fontsize)+"px; }</style>"))
    return None

def source(fn):
    import inspect
    print(inspect.getsource(fn))
    return None

import sys
# Adding the Winnie package's directory to your path
winnie_path = '../'

if winnie_path not in sys.path:
    sys.path.append(winnie_path)

##################
import numpy as np
import glob
import winnie
from winnie.plot import animate_quick_implot, quick_implot, mpl, plt
from spaceKLIP import database
import webbpsf
import astropy.units as u

setup_display()

### Input data:

The code assumes your data are processed through the imagetools.align_frames step from SpaceKLIP. I'd also strongly recommend coadding your integrations for each exposure before proceeding. The code will effectively do this anyway, but is much less careful about updating header info. For a spaceklip imagetools object called ImageTools, this is just:

> ImageTools.coadd_frames(nframes=None, types=['SCI', 'SCI_BG', 'REF', 'REF_BG'], subdir='coadded')


#### Notes on the current state of the code

- Many of the operations are written to optionally run on a GPU (by passing use_gpu=True to winnie.SpaceRDI). However, I'm still working through updating these functions in a few cases so the code will currently override this option with use_gpu=False.

- The convolution procedure (see tutorial 2) is appropriate only for round mask / Lyot data (NIRCam or MIRI) currently. Expect the wrong answer if you run this on FQPM data.

- winnie.SpaceRDI should propagate pixel-wise uncertainties from the ERR FITS extensions by default. However, a) these neglect speckle noise / speckle residuals, and b) the current SpaceKLIP ramp fitting with pseudo reference pixels significantly reduces noise but doesn't change the noise maps versus the standard pipeline — so these (generally) overestimate the noise. In other words, these are not statistically robust currently, but may still be useful for relative comparisons.

In [ ]:
# Prep the SpaceRDI object:
distance = 9.714 # Distance to your target in parsecs

base_dir = './aumic_rdi_example/'
input_dir = f'{base_dir}coadded/'
output_dir = f'{base_dir}WinnieRDI/'
data_ext = 'calints'
fitsfiles = np.sort(glob.glob(f'{input_dir}*{data_ext}.fits')) # Populate a file list

# Initialize the spaceKLIP database
Database = database.Database(output_dir)
Database.verbose = False
Database.read_jwst_s012_data(datapaths=fitsfiles,
                             psflibpaths=None,
                             bgpaths=None)

# Create the SpaceRDI object we'll use to carry out RDI
# There's quite a few options here. Check the docstring for more info!
wdb = winnie.SpaceRDI(Database, data_ext, overwrite=True, verbose=True, pad_data='auto')

In [ ]:
# Load the 0th concatenation (F356W for the example case). 
wdb.load_concat(0)

# Full concatenation strings can be used instead (the keys for Database.obs)

In [ ]:
# Run RDI using the default settings

rdi_reduc = wdb.run_rdi(save_products=True)

The run_rdi method returns a SpaceReduction object, which has the following properties:

    rdi_reduc.im: the derotated and roll-averaged final PSF-subtracted image (2D array)

    rdi_reduc.err: the propagated uncertainty maps based on the ERR fits extensions (2D array)

    rdi_reduc.rolls: the derotated and PSF-subtracted individual rolls (3D array)

    rdi_reduc.err_rolls: the propagated uncertainty maps for rdi_reduc.rolls (3D array)

    rdi_reduc.extent: the stellocentric image extent in arcsec for use with matplotlib

In [ ]:
# You can convert the output to a more familiar HDUList if you prefer:
hdul = rdi_reduc.to_hdulist()
hdul.info()

You can load a previous SpaceReduction object that you've saved in three ways:

By providing a) a SpaceRDI instance with a concat already loaded, and b) the output_ext used for the file:

`rdi_reduc = winnie.SpaceReduction(spacerdi=wdb, output_ext='rdi_psfsub')`

By providing a) a SpaceRDI instance with or without a concat already loaded, b) a concat to use, and c) the output_ext used for the file:

`rdi_reduc = winnie.SpaceReduction(spacerdi=wdb, concat=0, output_ext='rdi_psfsub')`

By providing the full path to the FITS file:

`rdi_reduc = winnie.SpaceReduction('./aumic_rdi_example/WinnieRDI/JWST_NIRCAM_NRCALONG_F356W_MASKA335R_SUB320A335R_rdi_psfsub.fits')`

In [ ]:
# Let's make a plot of the results, showing the combined image and each individual roll.

fig,axes,cbar = quick_implot([rdi_reduc.im, *rdi_reduc.rolls], show=False, clim_perc=99.5, cmap='RdBu_r', extent=rdi_reduc.extent, show_ticks=True, lims=[-6.5,6.5],
                              cbar=True, cbar_kwargs=dict(pad=0.02), panelsize=(5.5,4.5), cbar_label=r'MJy$\,/\,$sr')

for i,label in enumerate(['Combined', 'Roll 1', 'Roll 2']):
    axes[i].set_title(label)
    
plt.show()

Let's try running a different variant of RDI: RDI with high-pass filtered coefficients (HPFRDI). 

For any of the presets methods (rdi_presets, hpfrdi_presets, and mcrdi_presets), the output_ext argument can be used to change the suffix for the file when saved. By default, standard RDI is 'rdi_psfsub', HPFRDI is 'hpfrdi_psfsub', and MCRDI is 'mcrdi_psfsub'. Of course, you can also change this simply by assigning a new output_ext property.

Note: tutorial 3 covers what each of these presets is actually doing "under the hood". 

In [ ]:
# Set the HPFRDI presets and then rerun run_rdi to get our result
wdb.hpfrdi_presets()

hpfrdi_reduc = wdb.run_rdi(save_products=True)

In [ ]:
# Plot the comparison:

fig,axes,cbar = quick_implot([rdi_reduc.im, hpfrdi_reduc.im], show=False, clim_perc=99.9, cmap='RdBu_r', extent=rdi_reduc.extent, show_ticks=True, lims=[-6.5,6.5],
                              cbar=True, cbar_kwargs=dict(pad=0.02), panelsize=(5.5,4.5), cbar_label=r'MJy$\,/\,$sr')

for i,label in enumerate(['RDI', 'HPFRDI']):
    axes[i].set_title(label)
    
plt.show()

Running the same procedures on the 2nd concatenation (F444W)

In [ ]:
wdb.load_concat(1)

wdb.rdi_presets() # This is superfluous here, but included for reference — this just resets your RDI settings to the defaults
rdi_reduc = wdb.run_rdi(save_products=True)

wdb.hpfrdi_presets()
hpfrdi_reduc = wdb.run_rdi(save_products=True)

In [ ]:
# Plot the comparison:
fig,axes,cbar = quick_implot([rdi_reduc.im, hpfrdi_reduc.im], show=False, clim_perc=99.9, cmap='RdBu_r', extent=rdi_reduc.extent, show_ticks=True, lims=[-6.5,6.5],
                              cbar=True, cbar_kwargs=dict(pad=0.02), panelsize=(5.5,4.5), cbar_label=r'MJy$\,/\,$sr')

for i,label in enumerate(['RDI', 'HPFRDI']):
    axes[i].set_title(label)
    
plt.show()